In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import pickle
import psycopg2
import time
from datetime import datetime
import warnings
import math
import os
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore', category = FutureWarning)

In [ ]:
def postgresql_query(input_query, input_tuples = None):
    try:

        #new db 
        connection = psycopg2.connect(
            host="100.000.00.00",
            port = 5432,
            database="databse",
            user="user",
            password="password",
            # sslmode = "require"
        )

        cursor = connection.cursor()

        if input_tuples is not None:
            cursor.execute(input_query, input_tuples)
        else:
            cursor.execute(input_query)

        data = cursor.fetchall()

    except psycopg2.Error as e:
        print('Error connecting to the database:', e)
        return e

    else:
        if cursor:
            cursor.close()
        if connection:
            connection.close()
        
        return data

def get_option_data(table_name, mapped_days, lowest_STP, highest_STP):
    column_names = ['Date', 'Time', 'ExpiryDate', 'StrikePrice', 'Type', 'Open', 'High', 'Low', 'Close', 'OI', 'Volume', 'Ticker']
    option_data = pd.DataFrame(columns=column_names)
    
    for index in mapped_days.index:
        date = mapped_days.at[index, 'Date']
        expiry_date = mapped_days.at[index, 'ExpiryDate'] 
        # expiry_date = mapped_days.at[index, '2ndWeekExpiryDate']  # Adjust as needed

        query = f'''
            SELECT "Date", "Time", "ExpiryDate", "StrikePrice", "Type", "Open", "High", "Low", "Close", "OI", "Volume", "Ticker"
            FROM "{table_name}"
            WHERE "ExpiryDate" = %s AND ("Type" = 'PE' OR "Type" = 'CE') AND "Date" = %s AND "StrikePrice" BETWEEN {lowest_STP} AND {highest_STP}
        '''
        
        daily_data = postgresql_query(query, (expiry_date, date))

        # ✅ **Fix: Ensure daily_data is valid before converting to DataFrame**
        if daily_data and isinstance(daily_data, (list, tuple)) and len(daily_data) > 0:
            df = pd.DataFrame(daily_data, columns=column_names)
            option_data = pd.concat([option_data, df], ignore_index=True)
        else:
            print(f"⚠️ Warning: No data found for {date} (Expiry: {expiry_date})")
    
    option_data = option_data.drop_duplicates().reset_index(drop=True)
    
    return option_data




############for sensex########
# def get_option_data(table_name, mapped_days, lowest_STP, highest_STP):
#     column_names = ['Date', 'Time', 'ExpiryDate', 'StrikePrice', 'Type', 'Open', 'Ticker']
#     option_data = pd.DataFrame(columns=column_names)
    
#     for index in mapped_days.index:
#         date = mapped_days.at[index, 'Date']
#         expiry_date = mapped_days.at[index, 'ExpiryDate']
        
#         # Ensure expiry_date is a date object
#         if isinstance(expiry_date, str):
#             expiry_date = datetime.fromisoformat(expiry_date).date()
        
#         query = f'''
#             SELECT "Date", "Time", "ExpiryDate", "StrikePrice", "Type", "Open", "Ticker"
#             FROM "{table_name}"
#             WHERE "ExpiryDate"::DATE = %s AND ("Type" = 'PE' OR "Type" = 'CE') AND "Date" = %s AND "StrikePrice" BETWEEN {lowest_STP} AND {highest_STP}
#         '''
        
#         daily_data = postgresql_query(query, (expiry_date, date))
        
#         if daily_data:
#             df = pd.DataFrame(daily_data, columns=column_names)
#             option_data = pd.concat([option_data, df], ignore_index=True)
#         print(date)
    
#     option_data = option_data.drop_duplicates()
#     option_data = option_data.reset_index(drop=True)
    
#     return option_data



#all expiry data ######################
# def get_option_data(table_name, mapped_days, lowest_STP, highest_STP):
    
#     column_names = ['Date', 'Time', 'ExpiryDate', 'StrikePrice', 'Type', 'Open', 'Ticker']
#     option_data = pd.DataFrame(columns=column_names)
    
#     for index in mapped_days.index:
#         date = mapped_days.at[index, 'Date']
#         # expiry_date = mapped_days.at[index, 'ExpiryDate']  # Commented out if not needed

#         # Adjust the query based on whether ExpiryDate is needed or not
#         query = f'''
#             SELECT "Date", "Time", "ExpiryDate", "StrikePrice", "Type", "Open", "Ticker"
#             FROM "{table_name}"        
#             WHERE ("Type" = 'PE' OR "Type" = 'CE') AND "Date" = %s AND "StrikePrice" BETWEEN {lowest_STP} AND {highest_STP}
#         '''
        
#         # Execute the query with only the date parameter
#         daily_data = postgresql_query(query, (date,))
        
#         if daily_data:
#             df = pd.DataFrame(daily_data, columns=column_names)
#             option_data = pd.concat([option_data, df])
#         print(date)

#     option_data = option_data.drop_duplicates()
#     option_data = option_data.reset_index(drop=True)
    
#     return option_data


#####################

def get_dates(start_date, end_date):
    
    from datetime import datetime, timedelta
    
    # Edit Years and Months as needed
    # Here we are generating dates for the years 2022 to 2025 and all months from January to December.
    # You can adjust the years and months as per your requirement.
    years = [2022, 2023, 2024, 2025]
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    
    # Create a list of dates
    date_list = []
    
    for year in years:
        for month in months:
            # Find the number of days in the month
            days_in_month = (datetime(year, month % 12 + 1, 1) - timedelta(days=1)).day
    
            # Create a list of dates for each day in the month
            for day in range(1, days_in_month + 1):
                date_list.append(datetime(year, month, day))

    # # Print the list of dates
    # for date in date_list:
    #     print(date.strftime("%Y-%m-%d"))
    
    # Specify start and end dates
    # start_date = datetime(2023, 1, 1)
    # end_date = datetime(2023, 1, 10)
    
    # Filter the list of dates based on the specified range
    filtered_dates = [date.strftime("%Y-%m-%d") for date in date_list if start_date <= date <= end_date]
    
    # Print the filtered list of dates
    for date_str in filtered_dates:
        print(date_str)
        
    

In [ ]:
# Assuming postgresql_query and get_option_data functions are defined
stock = 'NIFTY'
max_range = 400 if stock == 'NIFTY' else (600 if stock == 'BANKNIFTY' or stock == 'SENSEX' else (600 if stock == 'FINNIFTY' else None))

# Uncomment the output_path based on the stock and give it the correct path
if stock == 'NIFTY':
    # output_path = rf"/home/newberry3/user/folder/"
    # output_path = rf"/home/newberry3/user/folder/"
    # output_path = "/home/newberry3/user/folder/"
    # output_path = "/home/newberry3/user/folder/"
elif stock =='BANKNIFTY':
    # output_path = rf"/home/newberry3/user/folder/" 
    # output_path = rf"/home/newberry3/user/folder/" 
elif stock =='FINNIFTY':
    # output_path = rf"/home/newberry3/user/folder/"   
    # output_path = rf"/home/newberry3/user/folder/" 
elif stock =='SENSEX':
    # output_path = rf"/home/newberry3/user/folder/"   
    # output_path = rf"/home/newberry3/user/folder/" 


# Here we are defining the Expiry file path based on the stocks and their respective expiry which will be used to map the expiry dates.

if stock =='BANKNIFTY':
    # expiry_file_path = "/home/newberry3/user/BANKNIFTY market dates.xlsx"
    # expiry_file_path = rf"/home/newberry3/user/BANKNIFTY market dates 2nd weekly.xlsx"

elif stock == 'NIFTY':
    # expiry_file_path = "/home/newberry3/user/NIFTY market dates.xlsx"
    # expiry_file_path = "/home/newberry3/user/NIFTY market dates 2nd weekly.xlsx"

elif stock == 'SENSEX':
    # expiry_file_path = "/home/newberry3/user/SENSEX market dates.xlsx"
    # expiry_file_path = "/home/newberry4/user/SENSEX market dates 2nd weekly.xlsx"

elif stock == 'FINNIFTY':
    # expiry_file_path =  "/home/newberry3/user/FINNIFTY_market_dates.xlsx"
    # expiry_file_path =  "/home/newberry3/user/FINNIFTY market dates 2nd weekly.xlsx"

# Edit Years and Months as needed
# Here we are generating dates for the years 2021 to 2025 and all months from January to December.
# You can adjust the years and months as per your requirement.
years = [2021, 2022, 2023, 2024, 2025]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Specify start and end dates 
start_date = '2021-06-01'
end_date = '2025-04-30'

for year in years:
    for month in months:
        
        date_list = []
        
        # Find the number of days in the month
        days_in_month = (datetime(year, month % 12 + 1, 1) - timedelta(days=1)).day

        # Create a list of dates for each day in the month
        for day in range(1, days_in_month + 1):
            date_list.append(datetime(year, month, day).strftime("%Y-%m-%d"))

        # Filter the list of dates based on the specified range
        filtered_dates = [date for date in date_list if start_date <= date <= end_date]

        if filtered_dates:
            start_date1 = filtered_dates[0]
            end_date1 = filtered_dates[-1]
            mapped_days = pd.read_excel(expiry_file_path)
            mapped_days = mapped_days[(mapped_days['Date'] >= start_date1) & (mapped_days['Date'] <= end_date1)]
            weekdays = mapped_days['Date'].to_list()
    
            # Pull Index data
            start_time = time.time()
            data = postgresql_query(f'''
                                    SELECT "Date", "Time", "Open", "High", "Low", "Close", "Type", "Ticker"
                                    FROM "{stock + '_IDX'}"
                                    WHERE "Type" = 'IDX'
                                    AND "Time" BETWEEN '09:15' AND '15:29'
                                    ''', (weekdays,))
            end_time = time.time()
            elapsed_time = end_time - start_time
            print('Time taken to get Index Data:', elapsed_time)
            
            column_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Type','Ticker']
            index_data = pd.DataFrame(data, columns = column_names)
            
            # Merge with ExpiryDate and change Ticker column to DateTime index
            df = index_data.merge(mapped_days, on = 'Date')
            
            df.set_index('Ticker', inplace = True)
            print(df)
            df.index = pd.to_datetime(df.index.str.replace(stock, ''), format = '%Y%m%d%H:%M')
            df = df.rename_axis('DateTime')
            df = df.sort_index()
            df = df.drop_duplicates()

            if stock == 'BANKNIFTY':
                lower_strike = round((df['Low'].min() - max_range) // 100) * 100
                higher_strike = round(math.ceil((df['High'].max() + max_range) / 100.0)) * 100
            elif stock == 'SENSEX':
                lower_strike = round((df['Low'].min() - max_range) // 100) * 100
                higher_strike = round(math.ceil((df['High'].max() + max_range) / 100.0)) * 100
            elif stock == 'FINNIFTY':
                lower_strike = round((df['Low'].min() - max_range) // 50) * 50
                higher_strike = round(math.ceil((df['High'].max() + max_range) / 50.0)) * 50
            elif stock == 'NIFTY':    
                lower_strike = 14800
                higher_strike = 30000

            table_name = stock + '_OPT_' + str(year)
            
            # Pull Options data day by day
            start_time = time.time()
            initial_option_data = get_option_data(table_name, mapped_days, lower_strike, higher_strike)
            initial_option_data['Ticker'] = initial_option_data['Ticker'].str.replace('-', '') 
            end_time = time.time()
            elapsed_time = end_time - start_time
            print('Time taken to pull Options Data:', elapsed_time)

            if month < 10:
                month = '0' + str(month)
                
            initial_option_data.to_pickle(output_path + f'{stock}_{year}{month}.pkl')
            print("done transferring data in ", output_path)


Time taken to get Index Data: 1.3942599296569824
                          Date   Time      Open      High       Low     Close  \
Ticker                                                                          
2021060109:15NIFTY  2021-06-01  09:15  15630.30  15630.30  15594.00  15613.55   
2021060109:16NIFTY  2021-06-01  09:16  15610.50  15618.20  15607.60  15609.45   
2021060109:17NIFTY  2021-06-01  09:17  15610.45  15611.10  15591.90  15599.85   
2021060109:18NIFTY  2021-06-01  09:18  15598.75  15610.05  15588.65  15606.85   
2021060109:19NIFTY  2021-06-01  09:19  15604.80  15606.40  15597.45  15602.95   
...                        ...    ...       ...       ...       ...       ...   
2021063015:25NIFTY  2021-06-30  15:25  15715.70  15716.60  15714.15  15714.15   
2021063015:26NIFTY  2021-06-30  15:26  15714.85  15719.70  15714.45  15719.70   
2021063015:27NIFTY  2021-06-30  15:27  15720.35  15726.35  15719.80  15726.25   
2021063015:28NIFTY  2021-06-30  15:28  15725.95  15727.40  1